In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json
import os
import tqdm
from langdetect import detect
from langdetect import DetectorFactory

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [5]:
root_path = '../Data'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str,
    'doi': str
})
meta_df.head()

C:\Users\JACKTANSNAKE\AppData\Local\Temp\ipykernel_14700\2995585893.py:3: DtypeWarning: Columns (1,5,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(metadata_path, dtype={


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [6]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056660 entries, 0 to 1056659
Data columns (total 19 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   cord_uid          1056660 non-null  object 
 1   sha               373766 non-null   object 
 2   source_x          1056660 non-null  object 
 3   title             1056157 non-null  object 
 4   doi               656780 non-null   object 
 5   pmcid             389571 non-null   object 
 6   pubmed_id         498932 non-null   object 
 7   license           1056660 non-null  object 
 8   abstract          821118 non-null   object 
 9   publish_time      1054846 non-null  object 
 10  authors           1032791 non-null  object 
 11  journal           969338 non-null   object 
 12  mag_id            0 non-null        float64
 13  who_covidence_id  482935 non-null   object 
 14  arxiv_id          14249 non-null    object 
 15  pdf_json_files    373766 non-null   object 
 16  

In [10]:
np.unique(meta_df['cord_uid']).shape

(970836,)

In [19]:
def get_dir_list(folder_dir):
    """
    Get all file names in a directory.
    :param folder_dir:
    :return:
    """
    dirlist = []
    directory = os.fsencode(folder_dir)

    for file in os.listdir(directory):
        filename = os.path.join(directory, file)
        filename = os.fsdecode(filename)
        dirlist.append(str(filename))

    return dirlist

In [20]:
pdf_json_list_dir = r'../Data/pdf_json'
pdf_json_list = get_dir_list(pdf_json_list_dir)

In [21]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(pdf_json_list[0])
print(first_row)

0000028b5cc154f68b8a269f6578f21e31f62977: ... According to current live statistics at the time of editing this letter, Russia has been the third country in the world to be affected by COVID-19 with both new cases and death rates rising. It remain...


In [22]:
from tqdm import tqdm
all_json_clean = list()
for idx, entry in tqdm(enumerate(pdf_json_list), total=len(pdf_json_list)):

    try:
        content = FileReader(entry)
    except Exception as e:
        continue  # invalid paper format, skip

    if len(content.body_text) == 0:
        continue

    all_json_clean.append(pdf_json_list[idx])

all_json = all_json_clean
len(all_json)

100%|██████████| 401212/401212 [29:51<00:00, 223.99it/s]


401212

In [23]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [25]:
from tqdm import tqdm
dict_ = {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in tqdm(enumerate(all_json), total = len(all_json)):

    try:
        content = FileReader(entry)
    except Exception as e:
        continue  # invalid paper format, skip

    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    if len(content.body_text) == 0:
        continue
    dict_['abstract'].append(content.abstract)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)

    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0:
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)

    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]

    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(get_breaks('. '.join(authors), 40))
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])

    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])

    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])

    # add doi
    dict_['doi'].append(meta_data['doi'].values[0])

df_covid = pd.DataFrame(dict_, columns=['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

100%|██████████| 401212/401212 [5:00:40<00:00, 22.24it/s]  


,paper_id,doi,abstract,body_text,authors,title,journal,abstract_summary
0,0000028b5cc154f68b8a269f6578f21e31f62977,10.1002/bjs.11940,,According to current live statistics at the ti...,"Tsarkov, Petr V.. Zubayraeva, Albina A.. <br...",“Multi‐faceted” COVID‐19: Russian<br>experience,Br J Surg,Not provided.
1,0000b6da665726420ab8ac9246d526f2f44d5943,10.1002/lio2.665,Objective: An at home-test for differentiating...,As a consequence of the global COVID-19 pandem...,"Yang, Alex. Lewis, Robert J.. Watson, Nora....",The cell phone vibration test: A telemedicine...,Laryngoscope Investig Otolaryngol,Objective: An at home-test for<br>differentia...
2,0000b93c66f991236db92dc16fa6db119b27ca12,10.1007/978-3-030-36358-1_7,,Infections are an important cause of morbidity...,"George, Biju. Bhattacharya, Sanjay",Infections in Hematopoietic Stem Cell<br>Tran...,Contemporary Bone Marrow Transplantation,Not provided.
3,0000fcce604204b1b9d876dc073eb529eb5ce305,10.1016/j.regg.2021.01.002,Contribución de los autores: Rocío Menéndez Co...,Se calcula que en España existen cerca de 367....,"Menéndez-Colino, Rocío. Argentina,<br>France...",La Geriatría de Enlace con residencias en la<...,Rev Esp Geriatr Gerontol,Contribución de los autores: Rocío Menéndez<b...
4,000122a9a774ec76fa35ec0c0f6734e7e8d0c541,10.1016/j.rec.2020.08.002,Introduction and objectives: The COVID-19 outb...,Introduction and objectives: The COVID-19 outb...,"Rodríguez-Leor, Oriol. Cid-Álvarez, Belén. <...",Impact of COVID-19 on ST-segment elevation<br...,Rev Esp Cardiol (Engl Ed),Introduction and objectives: The COVID-19<br>...


In [26]:
# df_covid.to_csv('../Data/raw.csv')

In [27]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349422 entries, 0 to 349421
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   paper_id          349422 non-null  object
 1   doi               337261 non-null  object
 2   abstract          349422 non-null  object
 3   body_text         349422 non-null  object
 4   authors           346559 non-null  object
 5   title             349418 non-null  object
 6   journal           318173 non-null  object
 7   abstract_summary  349422 non-null  object
dtypes: object(8)
memory usage: 21.3+ MB


In [ ]:
df_covid.head()